# Projeto 1 - Ciência dos Dados

Nome: Mateus Marinheiro

Nome: Henrique Albuquerque


Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [36]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [37]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\Mateus Marinhero\Documents\Insper\2023\2o Semestre\Ciência dos Dados\Projeto1\Projeto-1-C-Dados


Carregando a base de dados com os tweets classificados manualmente:

In [38]:
train = pd.read_excel('dados_treino.xlsx')
train.head(5)

,Mensagem,Target
0,Fique atento na compra. Ainda está com problem...,Relevante
1,Talvez a tradução antiga tivesse um pouco mais...,Relevante
2,"Sinceramente, também estou tentando entender c...",Relevante
3,Escolhi o livro por acreditar no tema e indica...,Irrelevante
4,"Recebi o produto avariado, no mesmo dia reclam...",Relevante


In [39]:
train.Target.value_counts()

Irrelevante    153
Relevante      147
Name: Target, dtype: int64

In [40]:
test = pd.read_excel('dados_teste.xlsx')
test.head(5)

,Mensagem,Target
0,"Não sou de desistir de livro, mas neste não te...",Irrelevante
1,EU NEM SE QUER RECEBI O PRODUTO! Voces são mui...,Relevante
2,Tao ruim quanto o primeiro. Fonte não justific...,Relevante
3,"Sou fã do canal, estava na primeira sessão da ...",Irrelevante
4,Péssimo. O autor claramente tem um visão disto...,Irrelevante


In [41]:
test.Target.value_counts()

Irrelevante    162
Relevante       38
Name: Target, dtype: int64

___
## Classificador automático de feedbacks de produtos da Amazon


Será criado um classificador segundo a teoria de probabilidade de Naïve Bayes, classificando notícias atuais sobre o mercado digital como:

- Relevante: Mensagens consideradas como relevantes serão aqueles que trazem alguma crítica à direcionadas a Amazon, sejam elas, em relação à preços, envios e outros problemas. Também serão considerados relevantes críticas feitas às editoras, sendo assim, mensagens criticando a tradução, formatação do texto, qualidade do produto entre outros.</li>
    
- Irrelevante: Mensagens consideradas como Irrelevantes serão aquelas que trazem qualquer tipo de opinião sobre o livro, qualquer tipo de elogio e qualquer outra coisa que não se enquadra como relevante.  </li>
    

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

#### 1 - Funções de  _cleanup_

Função que irá retirar das mensagens recebidas todos os ruídos, sendo eles, pountuações e símbolos, e também retiramos emojis, e números, que foram considerados irrelevantes para o nosso classificador.

In [42]:
# https://docs.python.org/3/library/re.html#
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[´"!-.:?;$'']' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

import re

def remove_emojis(text):
    # Define um padrão de regex para encontrar emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # Emoticons
                               u"\U0001F300-\U0001F5FF"  # Símbolos e pictogramas diversos
                               u"\U0001F680-\U0001F6FF"  # Transporte e símbolos de mapa
                               u"\U0001F700-\U0001F77F"  # Símbolos de alquimia, religião e zodíaco
                               u"\U0001F780-\U0001F7FF"  # Símbolos de traços diversos
                               u"\U0001F800-\U0001F8FF"  # Símbolos de escrita linear abugida
                               u"\U0001F900-\U0001F9FF"  # Símbolos suplementares de escrita linear abugida
                               u"\U0001FA00-\U0001FA6F"  # Símbolos de xadrez
                               u"\U0001FA70-\U0001FAFF"  # Símbolos de emojis adicionais
                               u"\U0001F004-\U0001F0CF"  # Símbolos de cartas de baralho
                               u"\U0001F170-\U0001F251"  # Símbolos de emojis de comemoração
                               "]+", flags=re.UNICODE)

    # Substitui os emojis por uma string vazia
    clean_text = emoji_pattern.sub(r'', text)

    return clean_text



#### 2 - Limpeza dos comentários

In [43]:
palavras_relevantes = []             #Lista com as palvras de feedbacks relevantes.
palavras_irrelevantes = []         #Lista com as palvras de feedbacks irrelevantes.

train_relevantes = train.loc[train.Target == 'Relevante', :]        #base de dados relevante
train_irrelevantes = train.loc[train.Target == 'Irrelevante', :]        #base de dados irrelevantes

for i in range(len(train_relevantes)):      #Limpando os emojis e potuação da base de dados relevante e colocando as palavras em uma lista.
    palavras_relevantes += cleanup(remove_emojis(train_relevantes.iloc[i, 0].lower())).split()

for i in range(len(train_irrelevantes)):    #Limpando os emojis e pontuação da base de dados irrelevante e colocando as palavras em uma lista.
    palavras_irrelevantes += cleanup(remove_emojis(train_irrelevantes.iloc[i, 0].lower())).split()

#Tirando númreros das listas de palavras.

for i in range(len(palavras_relevantes)):
    for caractere in palavras_relevantes[i]:
        if caractere.isdigit():
            palavras_relevantes[i] = ''
    
for i in range(len(palavras_irrelevantes)):
    for caractere in palavras_irrelevantes[i]:
        if caractere.isdigit():
            palavras_irrelevantes[i] = ''

#Criando lista com todas as palavras
total_palavras = palavras_relevantes + palavras_irrelevantes


#### 3 - Criando pd.series para cada Target

In [44]:
#Criando Série para palavras relevantes
serie_relevantes = pd.Series(palavras_relevantes)

#Criando Série para palavras não relevantes
serie_irrelevantes = pd.Series(palavras_irrelevantes)

#Criando série para todas as palavras:
serie_total = pd.Series(total_palavras)

#### 4 - Gerando tabela de frequências para as Séries

In [45]:
#Tabela de frequência absoluto de palavras relevantes
tabela_relevante_absoluto = serie_relevantes.value_counts()
#Tabela de frequência relativa de palavras relevantes
tabela_relevante_frequencia = serie_relevantes.value_counts(True)


#Tabela de frequência absoluta de palavras não relevantes
tabela_irrelevante_absoluto = serie_irrelevantes.value_counts()
#Tabela de frequência relativa de palavras não relevantes
tabela_irrelevante_frequencia = serie_irrelevantes.value_counts(True)


#Tabela de frequência absoluta de todas as palavras
tabela_total_absoluto = serie_total.value_counts()
#Tabela de frequência relativa de todas as palavras
tabela_total_frequencia = serie_total.value_counts(True)


#### 5 - Suavilização de Laplace


In [46]:
def laplace_relevante (comentario):
    if len(comentario) > 0:
        palavras = comentario
    P_i = 1
    for i in palavras:
        P_i *=  (tabela_relevante_absoluto[i] + 1) / (tabela_relevante_absoluto.sum() + len(tabela_total_absoluto))

    return P_i

def laplace_irrelevante (comentario):
    if len(comentario) > 0:
        palavras = comentario
    P_i = 1
    for i in palavras:
        P_i *=  (tabela_irrelevante_absoluto[i] + 1) / (tabela_irrelevante_absoluto.sum() + len(tabela_total_absoluto))

    return P_i


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

#### Teoria

Naive Bayes é um algoritmo de classificação probabilístico que utiliza o Teorema de Bayes para inferir a classe de uma amostra. Ele é frequentemente usado na área de Data Science para categorizar textos com base na frequência de palavras que aparecem nos mesmos. A grande vantagem do Naive Bayes é a sua simplicidade e a forma "ingênua" como ele trata as variáveis, assumindo que elas são independentes entre si.

Para utilizar o algoritmo Naive Bayes na programação, é necessário seguir algumas etapas. Primeiro, são definidas algumas probabilidades como a probabilidade de um comentário em produtos da Amazon ser, ou relevante (como definimos anteriormente), ou irrelevante (como definido anteriormente também), bem como a probabilidade de um comentário ser classificado como um desses Targets, dada que foi classificada como uma delas. Em seguida, são calculadas essas probabilidades, utilizando técnicas como a suavização de Laplace para evitar casos em que uma palavra não esteja na base de dados.

Para calcular a probabilidade de um comentário ser classificado como algum dos Targets, é necessário contar o número de palavras dos comentários classificados como esse Target e dividir pelo número total de palavras de todas as notícias.

Por fim, para inferir a classe de uma amostra, é necessário calcular as probabilidades condicionais utilizando o Teorema de Bayes e escolher a classe com a maior probabilidade. É importante lembrar que o Naive Bayes assume que as variáveis são independentes entre si, o que pode levar a uma classificação incorreta em alguns casos. <strong>Segue abaixo um pouco do raciocínio matemático: </strong>

###### Definindo algumas probabilidades:
- $P(Comentário|Relevante)$: probabilidade de encontrar o comentário dado as palavras Relevantes;
- $P(Comentário|Irrelevante)$: probabilidade de encontrar o comentário dado as palavras Irrelevantes;
- $P(Relevante)$: probabilidade do comentário ser Relevante;
- $P(Irrelevante)$: probabilidade do comentário ser Irrelevante;
- $P(Comentário)$: probabilidade do comentário ocorrer.


$$P(Relevante|Comentário) = \frac{P(Comentário|Relevante) P(Relevante)}{P(Comentário)}$$

<br>

$$P(Irrelevante|Comentário) = \frac{P(Comentário|Irrelevante) P(Irrelevante)}{P(Comentário)}$$

<br>

Primeiramente vamos calcular as seguintes probabilidades: $P(Relevante)$, $P(Irrelevante)$.


- Número de palavras dos comentários relevantes: $Nr$
- Número de palavras dos comentários irrelevantes: $Ni$
- Número total de palavras: $Nt$

<br>

$$P(Relevante) = \frac{Nr}{Nt}$$

<br>

<br>

$$P(Irrelevante) = \frac{Ni}{Nt}$$

<br>


Feito isso, vamos calcular as seguinte probabilidades: $P(Comentário|Relevante)$ , $P(Comentário|Irrelevante)$.

Como calcular essa probabiilidades? Será necessário dividir as palavras de um comentário, e calcular a frequência absoluta de vezes que essa palavra aparece no total de palavras em cada Target, e dessa maneira, dividir pelo valor total de palavras que estão no conjunto de palavras dos comentários. Após feito isso é necessário multiplicar o valor obtido para cada palavra e assim temos a probabilidade $P()$ para um determinado comentário.

Caso a palavras não esteja na nossa base de dados, utilizaremos a suavização de Laplace para prevenir casos como esse.

### Suavização de Laplace:
Esse método é utilizado para evitar que a probabilidade resulte em 0.
Vamos mostrar isso com ajuda da matemática:

- Se queremos calcular a probabilidade do comentário ser relevante dado que o comentário, porém, uma das palavras contidas nesse comentário não está na nossa base de dados. Usaremos portanto a suavização de Laplace.

$$P(Palavra|Relevante) = \frac{0}{Nr}$$

Para que isso não aconteça nós somaremos 1 no numerador e para manter o equilíbrio somaremos a quantidade total de possíveis palavras da nossa base de dados:

- Número total de possíveis palavras da nossa base de dados: $Npp$


$$P(Palavra|Relevante) = \frac{0 + 1}{Nr + Npp}$$

<br>

Sendo assim, nós teremos uma fórmula geral descrita da forma:

- Frequência absoluta da palavra na lista de comentários relevantes: $Far$
- Frequência absoluta da palavra na lista de comentários irrelevantes: $Fai$

<br>

$$P(Palavra|Relevante) = \frac{Far + 1}{Nr + Npp}$$

<br>

$$P(Palavra|Irrelevante) = \frac{Fai + 1}{Ni + Npp}$$

<br>

Com as duas fórmulas gerais, agora será necesséerio calcular a probabilidade de cada palavra, dado cada um dos dois Targetes, e multiplicar esses valores para obter a probabilidade de um comentário ser classificado como um dos dois Targets.
Assim teremos, $P(Comentário|Relevante)$ e $P(Comentário|Irrelevante)$, das seguintes fórmulas:

$$P(Relevante|Comentário) = \frac{P(Comentário|Relevante) P(Relevante)}{P(Comentário)}$$

<br>

$$P(Irrelevante|Comentário) = \frac{P(Comentário|Irrelevante) P(Irrelevante)}{P(Comentário)}$$

<br>

Como o denominador é o mesmo para as duas fórmulas, podemos simplificar e obter a seguinte fórmula:

$$P(Relevante|Comentário) = P(Comentário|Relevante) P(Relevante)$$

<br>

$$P(Irrelevante|Comentário) = P(Comentário|Irrelevante) P(Irrelevante)$$

<br>

Agora, para inferir a classe de uma amostra, é necessário calcular as probabilidades condicionais utilizando o Teorema de Bayes e escolher a classe com a maior probabilidade. É importante lembrar que o Naive Bayes assume que as variáveis são independentes entre si, o que pode levar a uma classificação incorreta em alguns casos.



#### 1 - Criando um coluna no nosso dataframe de teste para guardar os resultados do nosso classificador

In [47]:
#Criando coluna 'Classificador' na base de dados de teste
test['Classificador'] = ''

#### 2 - Classificando os comentários do dataframe de teste

In [48]:
P_R = len(palavras_relevantes)/len(total_palavras)                                                                              # Probabilidade de ser relevante
P_I = len(palavras_irrelevantes)/len(total_palavras)                                                                            # Probabilidade de ser irrelevante

for i in range(len(test.Mensagem)):                                                                                             # Loop para percorrer todas as linhas da base de dados de teste
    comentario = test.iloc[i,0]                                                                                                 # Comentário a ser classificado
    comentario_limpo = cleanup(remove_emojis(comentario.lower())).split()                                                       # Comentário limpo
    produto_prob_R = 1                                                                                                          # Variável que armazena o produto das probabilidades de ser relevante
    produto_prob_I = 1                                                                                                          # Variável que armazena o produto das probabilidades de ser irrelevante
    for palavra in comentario_limpo:                                                                                            # Loop para percorrer todas as palavras do comentário
        
        if palavra not in palavras_relevantes:                                                                                  # Se a palavra não estiver na lista de palavras relevantes, adiciona ela com frequência 0, necessária para a suavização de Laplace
            tabela_relevante_absoluto[palavra] = 0                  
        
        if palavra not in palavras_irrelevantes:                                                                                # Se a palavra não estiver na lista de palavras irrelevantes, adiciona ela com frequência 0, necessária para a suavização de Laplace
            tabela_irrelevante_absoluto[palavra] = 0
            

        produto_prob_R *= (tabela_relevante_absoluto[palavra] + 1)/(len(palavras_relevantes) + len(tabela_total_absoluto))      # Calculando o produto das probabilidades de ser relevante
        produto_prob_I *= (tabela_irrelevante_absoluto[palavra] + 1)/(len(palavras_irrelevantes) + len(tabela_total_absoluto))  # Calculando o produto das probabilidades de ser irrelevante
        

    probabilidade_relevante = produto_prob_R * P_R                                                                              # Calculando a probabilidade de ser relevante
    probabilidade_irrelevante = produto_prob_I * P_I                                                                            # Calculando a probabilidade de ser irrelevante

    if probabilidade_relevante > probabilidade_irrelevante:                                                                     # Se a probabilidade de ser relevante for maior que a probabilidade de ser irrelevante, classifica como relevante
        test.iloc[i,2] = 'Relevante'                                                                                            # Classifica como relevante
    else:                                                                                                                       # Se a probabilidade de ser irrelevante for maior que a probabilidade de ser relevante, classifica como irrelevante
        test.iloc[i,2] = 'Irrelevante'                                                                                          # Classifica como irrelevante



#### 3 - Verificando a performance do nosso classificador

Coluna 'Target' são as classificações feitas manualmente

Coluna 'Classificação' são as classificações feitas pelo nosso classificador

In [49]:
test

,Mensagem,Target,Classificador
0,"Não sou de desistir de livro, mas neste não te...",Irrelevante,Irrelevante
1,EU NEM SE QUER RECEBI O PRODUTO! Voces são mui...,Relevante,Relevante
2,Tao ruim quanto o primeiro. Fonte não justific...,Relevante,Relevante
3,"Sou fã do canal, estava na primeira sessão da ...",Irrelevante,Irrelevante
4,Péssimo. O autor claramente tem um visão disto...,Irrelevante,Irrelevante
...,...,...,...
195,lixo de um astrólogo limítrofe,Irrelevante,Irrelevante
196,Achei ridículo! Eu realmente esperava uma hist...,Irrelevante,Irrelevante
197,"As afirmações do autor, que é historiador, não...",Irrelevante,Irrelevante
198,"Livro meio fantasioso , quer dizer que quase n...",Irrelevante,Irrelevante


In [50]:
pd.crosstab(test.Target, test.Classificador, normalize = 'index').round(6)*100

Classificador,Irrelevante,Relevante
Target,,
Irrelevante,93.8272,6.1728
Relevante,10.5263,89.4737


Porcentagem dos comentários relevantes classificados como relevantes: 89.5%

Porcentagem dos comentários relevantes classificados como irrelevantes: 10.5%

Porcentagem dos comentários irrelevantes classificados como relevantes: 6.2%

Porcentagem dos comentários irrelevantes classificados como irrelevantes: 93.8%

In [51]:
#Acurácia do nosso classificador:

acuracia = ((pd.crosstab(test.Target, test.Classificador, normalize = 'index').round(6)*100).iloc[0,0] + (pd.crosstab(test.Target, test.Classificador, normalize = 'index').round(6)*100).iloc[1,1])/2
print('A acurácia do nosso classificador é de {}%'.format(acuracia))


A acurácia do nosso classificador é de 91.65045%


___
### Concluindo

#### Após a análise do nosso classificador, podemos concluir que:

Porcentagem de verdadeiros relevantes: 89.5%

Porcentagem de falsos relevantes: 10.5%

Porcentagem de verdadeiros irrelevantes: 93.8%

Porcentagem de falsos irrelevantes: 6.2%

O classificador teve uma acurácia de 91.6% na classificação dos comentários.


**Com base nessa porcentagens podemos concluir que o classficador gerou bons resultados como uma primeira hipótese, porém, ainda há espaço para melhorias, já que sua acurácia não é de 100%.**

#### Por quê não usar o próprio classificador para gerar mais amostras de treinamento?

Não podemos usar o próprio classificador para gerar mais amostras de treinamento, pois o classificador não ia criar palavras novas, portanto ia criar mais frases com as mesmas palavras, somente diferentes combinações, e isso não iria ajudar a melhorar o classificador, pois não iria criar frases genéricas.

#### Diferentes cenários para o classificador Naive-Bayes

1. O classificador Naive-Bayes pode ser utilizado para classficar qual time uma pessoa torçe com base em comentários feitos em redes sociais a respeito de futebool, certas palavras chaves podem indicar qual time a pessoa torçe, como por exemplo, "Flamengo" e "Mengão" indicam que a pessoa torçe para o Flamengo, e assim por diante.

2. O classificador Naive-Bayes, também pode ser utilizado para classificar artigos públicados, como por exemplo, artigos de opinião, artigos de notícias, artigos de entretenimento, artigos de esportes, etc. Para isso, basta criar uma base de dados com palavras chaves que indicam cada tipo de artigo, e assim, o classificador poderá classificar os artigos de acordo com as palavras chaves contidas neles.

#### Melhorias reais com indicações concretas de como implementar

1. Uma das inúmeras melhorias que podem ser aplicadas ao nosso classificador é a utilização de uma base de dados maior, com mais comentários, pois assim, o classificador terá mais palavras para trabalhar, e assim, terá uma maior acurácia.

2. Outra melhoria que pode ser aplicada é uma limpeza mais abrangente de palavras e elementos irrelevantes para os comentários, assim teriamos uma base de dados mais limpa, com mais palvras-chaves e menos ruídos. Por exemplo, podem ser utilizados, lemmatization, que é a redução de palavras flexionadas à sua forma base, ou seja, ao lema. Por exemplo, a palavra "correria" seria reduzida para "correr", e assim por diante. Outra técnica que pode ser utilizada é a remoção de stopwords, que são palavras que não possuem significado, como por exemplo, "a", "o", "de", "para", etc. Essas técnicas foram apresentadas na rúbrica e mais aprofundada no site Geeks for Geeks.



___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

In [52]:
#Base de dados com todos os comentários
dados_totais = pd.concat([train, test], ignore_index = True)
dados_totais

,Mensagem,Target,Classificador
0,Fique atento na compra. Ainda está com problem...,Relevante,NaN
1,Talvez a tradução antiga tivesse um pouco mais...,Relevante,NaN
2,"Sinceramente, também estou tentando entender c...",Relevante,NaN
3,Escolhi o livro por acreditar no tema e indica...,Irrelevante,NaN
4,"Recebi o produto avariado, no mesmo dia reclam...",Relevante,NaN
...,...,...,...
495,lixo de um astrólogo limítrofe,Irrelevante,Irrelevante
496,Achei ridículo! Eu realmente esperava uma hist...,Irrelevante,Irrelevante
497,"As afirmações do autor, que é historiador, não...",Irrelevante,Irrelevante
498,"Livro meio fantasioso , quer dizer que quase n...",Irrelevante,Irrelevante


In [54]:
#Apagando a coluna 'Classificador' da base de dados com todos os comentários
dados_totais.drop('Classificador', axis= 1 , inplace = True)
dados_totais

,Mensagem,Target
0,Fique atento na compra. Ainda está com problem...,Relevante
1,Talvez a tradução antiga tivesse um pouco mais...,Relevante
2,"Sinceramente, também estou tentando entender c...",Relevante
3,Escolhi o livro por acreditar no tema e indica...,Irrelevante
4,"Recebi o produto avariado, no mesmo dia reclam...",Relevante
...,...,...
495,lixo de um astrólogo limítrofe,Irrelevante
496,Achei ridículo! Eu realmente esperava uma hist...,Irrelevante
497,"As afirmações do autor, que é historiador, não...",Irrelevante
498,"Livro meio fantasioso , quer dizer que quase n...",Irrelevante


In [55]:
#Embaralhando a base de dados com todos os comentários
dados_totais = dados_totais.sample(frac = 1).reset_index(drop = True)
dados_totais

,Mensagem,Target
0,Não consegui terminar a leitura. Achei superfi...,Irrelevante
1,O Ebook esta bem mais caro que o livro físico....,Relevante
2,Como disseram em outra avaliação o livro dever...,Irrelevante
3,"O livro é lindo! Edição maravilhosa. Mas, demo...",Irrelevante
4,Oh Amazon! Por favor atualiza essa edição! Est...,Relevante
...,...,...
495,Não consigo ler .... Está muito confuso este s...,Relevante
496,Apesar do enorme apreço que tenho por Mário S....,Irrelevante
497,As ilustrações da capa dos livros são de fato ...,Relevante
498,Minha experiência com ebook foi frustrante... ...,Relevante


In [57]:
#Separando a base de dados em 2 bases de dados, uma com 300 dos comentários e outra com 200 dos comentários
dados_treino_2 = dados_totais.iloc[:300, :]
dados_teste_2 = dados_totais.iloc[300:, :]
dados_treino_2




,Mensagem,Target
0,Não consegui terminar a leitura. Achei superfi...,Irrelevante
1,O Ebook esta bem mais caro que o livro físico....,Relevante
2,Como disseram em outra avaliação o livro dever...,Irrelevante
3,"O livro é lindo! Edição maravilhosa. Mas, demo...",Irrelevante
4,Oh Amazon! Por favor atualiza essa edição! Est...,Relevante
...,...,...
295,Esperava mais... pra escrever isso preferia te...,Irrelevante
296,Pra mim isso nem pode ser chamado de livro. A ...,Irrelevante
297,Porque não estão vendendo a versão em e-book k...,Relevante
298,"Pessoal, vocês vão me desculpar, mas a resenha...",Irrelevante


In [58]:
dados_teste_2

,Mensagem,Target
300,"O livro é muito repetitivo no começo, o que to...",Irrelevante
301,Quer bancar o diferente e só consegue ser prec...,Irrelevante
302,"Amei, como toas obras da Lanni. Escrita perfei...",Irrelevante
303,"Incrível como um e-book que custa US$9,99 na a...",Relevante
304,O preço da versão digital deste livro é injust...,Relevante
...,...,...
495,Não consigo ler .... Está muito confuso este s...,Relevante
496,Apesar do enorme apreço que tenho por Mário S....,Irrelevante
497,As ilustrações da capa dos livros são de fato ...,Relevante
498,Minha experiência com ebook foi frustrante... ...,Relevante


___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**